In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from keras.utils.vis_utils import plot_model
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

CONCATENO I FILE

In [ ]:
# caricare a mano solo file options, no underlying
df = []
for file in os.listdir('/'):
  if file.endswith('csv'):
    print('Loading file {0}...'.format(file))
    df.append(pd.read_csv(os.path.join('/', file), sep = ';'))

In [ ]:
df = pd.concat(df, axis = 0)
df.to_csv('options.csv', index = False)

In [ ]:
print(df)

In [ ]:
df = df.drop(columns=['Expiry_Date','Exercise_Style','Implied_Volatility','Sigma60'])

In [ ]:
df = df.dropna(axis=0)
df.shape

(218863, 8)

In [ ]:
call_df = df[df.Flag == 'Call'].drop(['Flag'], axis=1)
put_df = df[df.Flag == 'Put'].drop(['Flag'], axis=1)
print(call_df)

In [ ]:
# solo ora caricare file underlying
underlying = pd.read_csv('/underlying.csv',sep=';')
print(underlying)

In [ ]:
N_TIMESTEPS = 60

In [ ]:
padded = np.insert(underlying.Close.values, 0, np.array([np.nan] * N_TIMESTEPS))

In [ ]:
rolled = np.column_stack([np.roll(padded, i) for i in range(N_TIMESTEPS)])

In [ ]:
rolled = rolled[~np.isnan(rolled).any(axis=1)]

In [ ]:
rolled = np.column_stack((underlying.Date.values[N_TIMESTEPS - 1:], rolled))

In [ ]:
price_history = pd.DataFrame(data=rolled)

In [ ]:
joined = df.join(price_history.set_index(0), on='Date')
print(joined)

In [ ]:
call_df = joined[joined.Flag == 'Call'].drop(['Flag'], axis=1)
put_df = joined[joined.Flag == 'Put'].drop(['Flag'], axis=1)
call_df = call_df.drop(columns=['Date'])
put_df = put_df.drop(columns=['Date'])
print(call_df)

In [ ]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['Bid', 'Ask'], axis=1), 
                                                                        call_df[['Bid','Ask']],
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['Bid', 'Ask'], axis=1),
                                                                    put_df[['Bid','Ask']],
                                                                    test_size=0.01, random_state=42)
print(call_X_train)

In [ ]:
call_X_train = call_X_train.values
call_X_test = call_X_test.values
put_X_train = put_X_train.values
put_X_test = put_X_test.values

In [ ]:
call_X_train = [call_X_train[:, -N_TIMESTEPS:].reshape(call_X_train.shape[0], N_TIMESTEPS, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -N_TIMESTEPS:].reshape(call_X_test.shape[0], N_TIMESTEPS, 1), call_X_test[:, :4]]
put_X_train = [put_X_train[:, -N_TIMESTEPS:].reshape(put_X_train.shape[0], N_TIMESTEPS, 1), put_X_train[:, :4]]
put_X_test = [put_X_test[:, -N_TIMESTEPS:].reshape(put_X_test.shape[0], N_TIMESTEPS, 1), put_X_test[:, :4]]

In [ ]:
layers = 4 
features = 4
n_batch = 128 
n_epochs = 40

In [ ]:
def make_model():
    close_history = Input((N_TIMESTEPS, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=8, input_shape=(N_TIMESTEPS, 1), return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(400, kernel_initializer=tf.keras.initializers.GlorotNormal())(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(2, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

In [ ]:
call_model = make_model()

In [ ]:
call_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 8)            1952        ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 12)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

In [ ]:
call_model.compile(optimizer=Adam(lr=1e-2), loss='mse')
history = call_model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
call_model.save('saved-models/20221003-call-lstm-v1.h5')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/40
827/827 [==============================] - 24s 17ms/step - loss: 11718.6670 - val_loss: 10042.2646
Epoch 2/40
827/827 [==============================] - 12s 15ms/step - loss: 3069.8538 - val_loss: 2126.4910
Epoch 3/40
827/827 [==============================] - 12s 15ms/step - loss: 2994.2952 - val_loss: 4591.5747
Epoch 4/40
827/827 [==============================] - 12s 15ms/step - loss: 2963.5598 - val_loss: 506.6681
Epoch 5/40
827/827 [==============================] - 12s 15ms/step - loss: 2727.4934 - val_loss: 12287.1973
Epoch 6/40
827/827 [==============================] - 12s 15ms/step - loss: 3151.8442 - val_loss: 8980.4824
Epoch 7/40
827/827 [==============================] - 12s 15ms/step - loss: 2569.3711 - val_loss: 358.2010
Epoch 8/40
827/827 [==============================] - 12s 15ms/step - loss: 2905.1638 - val_loss: 2748.6440
Epoch 9/40
827/827 [==============================] - 12s 15ms/step - loss: 2634.9792 - val_loss: 1476.4606
Epoch 10/40
827/827 [======

In [ ]:
call_model.compile(optimizer=Adam(lr=1e-3), loss='mse')
history = call_model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=20, # notare i 20 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
call_model.save('saved-models/20221003-call-lstm-v2.h5')

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


827/827 [==============================] - 19s 16ms/step - loss: 1316.4855 - val_loss: 137.0657
Epoch 2/20
827/827 [==============================] - 12s 15ms/step - loss: 1404.6653 - val_loss: 431.0974
Epoch 3/20
827/827 [==============================] - 12s 15ms/step - loss: 1287.7406 - val_loss: 106.9891
Epoch 4/20
827/827 [==============================] - 12s 15ms/step - loss: 1291.8506 - val_loss: 207.0080
Epoch 5/20
827/827 [==============================] - 12s 15ms/step - loss: 1302.3651 - val_loss: 111.0294
Epoch 6/20
827/827 [==============================] - 12s 15ms/step - loss: 1337.7760 - val_loss: 117.3883
Epoch 7/20
827/827 [==============================] - 12s 15ms/step - loss: 1367.4969 - val_loss: 150.7468
Epoch 8/20
827/827 [==============================] - 12s 15ms/step - loss: 1314.4283 - val_loss: 455.5720
Epoch 9/20
827/827 [==============================] - 12s 15ms/step - loss: 1313.0155 - val_loss: 394.8969
Epoch 10/20
827/827 [===========================

In [ ]:
call_model.compile(optimizer=Adam(lr=1e-4), loss='mse')
history = call_model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=5, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
call_model.save('saved-models/20221003-call-lstm-v3.h5')

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


827/827 [==============================] - 20s 17ms/step - loss: 1678.7850 - val_loss: 96.6513
Epoch 2/5
827/827 [==============================] - 13s 16ms/step - loss: 1665.2687 - val_loss: 107.1808
Epoch 3/5
827/827 [==============================] - 13s 16ms/step - loss: 1589.6492 - val_loss: 129.9629
Epoch 4/5
827/827 [==============================] - 13s 15ms/step - loss: 1525.7357 - val_loss: 107.7833
Epoch 5/5
827/827 [==============================] - 13s 15ms/step - loss: 1609.1389 - val_loss: 133.2945


In [ ]:
put_model = make_model()

In [ ]:
put_model.compile(optimizer=Adam(lr=1e-2), loss='mse')
history = put_model.fit(put_X_train, put_y_train, 
                    batch_size=n_batch, epochs=n_epochs,
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
put_model.save('saved-models/20221003-put-lstm-v1.h5')

Epoch 1/40


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


849/849 [==============================] - 20s 17ms/step - loss: 14707.5986 - val_loss: 4092.4236
Epoch 2/40
849/849 [==============================] - 13s 15ms/step - loss: 3211.3591 - val_loss: 1475.2931
Epoch 3/40
849/849 [==============================] - 13s 15ms/step - loss: 3165.6074 - val_loss: 13015.1934
Epoch 4/40
849/849 [==============================] - 13s 15ms/step - loss: 2853.0225 - val_loss: 59380.4492
Epoch 5/40
849/849 [==============================] - 13s 15ms/step - loss: 2826.6267 - val_loss: 2412.1211
Epoch 6/40
849/849 [==============================] - 13s 15ms/step - loss: 3115.9810 - val_loss: 8996.1250
Epoch 7/40
849/849 [==============================] - 13s 15ms/step - loss: 2881.3853 - val_loss: 50121.0039
Epoch 8/40
849/849 [==============================] - 13s 15ms/step - loss: 2871.6001 - val_loss: 4985.3730
Epoch 9/40
849/849 [==============================] - 13s 15ms/step - loss: 2551.4473 - val_loss: 6202.7329
Epoch 10/40
849/849 [==============

In [ ]:
put_model.compile(optimizer=Adam(lr=1e-3), loss='mse')
history = put_model.fit(put_X_train, put_y_train, 
                    batch_size=n_batch, epochs=20,
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
put_model.save('saved-models/20221003-put-lstm-v2.h5')

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


849/849 [==============================] - 19s 16ms/step - loss: 1422.8582 - val_loss: 1083.5237
Epoch 2/20
849/849 [==============================] - 12s 15ms/step - loss: 1478.2268 - val_loss: 413.7445
Epoch 3/20
849/849 [==============================] - 12s 15ms/step - loss: 1513.4573 - val_loss: 348.2049
Epoch 4/20
849/849 [==============================] - 13s 15ms/step - loss: 1252.2172 - val_loss: 198.9076
Epoch 5/20
849/849 [==============================] - 13s 15ms/step - loss: 1502.6931 - val_loss: 240.7350
Epoch 6/20
849/849 [==============================] - 13s 15ms/step - loss: 1470.5720 - val_loss: 410.0989
Epoch 7/20
849/849 [==============================] - 13s 15ms/step - loss: 1517.8636 - val_loss: 484.1372
Epoch 8/20
849/849 [==============================] - 13s 15ms/step - loss: 1385.7773 - val_loss: 89.5529
Epoch 9/20
849/849 [==============================] - 13s 15ms/step - loss: 1405.8359 - val_loss: 202.8044
Epoch 10/20
849/849 [===========================

In [ ]:
from google.colab import files
files.download('saved-models/20221003-call-lstm-v1.h5')
files.download('saved-models/20221003-call-lstm-v2.h5')
files.download('saved-models/20221003-put-lstm-v1.h5')
files.download('saved-models/20221003-put-lstm-v2.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>